In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from bayes_optimization import BayesianOptimization

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess data
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
y_train = OneHotEncoder(sparse=False).fit_transform(y_train.reshape(-1, 1))
y_test = OneHotEncoder(sparse=False).fit_transform(y_test.reshape(-1, 1))

# Split data into training and validation sets
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.2, random_state=42)

# Define Bayesian optimization space
def optimize_hyperparameters(lr, epochs):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(784,)))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, verbose=0)
    return -model.evaluate(X_test, y_test, verbose=0)[1]  # Use -accurcy as the objective function

# Perform Bayesian optimization
bo = BayesianOptimization(f=optimize_hyperparameters, pbounds={'lr': (1e-5, 1e-1), 'epochs': (1, 100)}, random_state=42)
bo.maximize(init_points=5, n_iter=10)
print('Optimal hyperparameters: ', bo.res['params'])